# Bài tập 2. 
Cài đặt mô phỏng hàng đợi trong printer spool trong LAN. Các máy cần in sẽ gửi yêu cầu job tới máy in và máy in sẽ xử lý lần lượt theo thứ tự. Cần mô phỏng xử lý của máy in với các trường hợp
+ Bộ nhớ lớn, có thể queue được nhiều yêu cầu in
+ Bộ nhớ nhỏ, chỉ queue được cỡ 10 yêu cầu in (các job đến sau mà quá 10 sẽ tự động bị drop)
Máy in giả sử in mỗi bản in tầm 3-5s, mô phỏng các job xử lý với yêu cầu in có số trang in random từ 1-50 trang


# Tư duy Triển khai: Mô phỏng Hệ thống Printer Spooler (LAN)

## 1. Bản chất bài toán: Producer-Consumer (Nhà sản xuất - Người tiêu dùng)
Bài toán này là ví dụ kinh điển minh họa sự chênh lệch tốc độ xử lý trong hệ thống máy tính.
* **Producer (Người sản xuất):** Các máy tính (User) trong mạng LAN. Tốc độ tạo ra yêu cầu in (Print Job) rất nhanh (ví dụ: 0.5s/job).
* **Consumer (Người tiêu dùng):** Máy in (Printer). Tốc độ xử lý chậm (mất 3-5s cho một Job).
* **Buffer (Vùng đệm):** Hàng đợi (Queue/Spooler). Nơi chứa các Job chưa kịp xử lý để trung hòa tốc độ giữa hai bên.

## 2. Thách thức kỹ thuật & Giải pháp

### A. Vấn đề bất đồng bộ (Asynchronous)
* **Thực tế:** Nếu viết code tuần tự (Sequential), khi máy in đang chạy (sleep 5s), toàn bộ chương trình sẽ dừng lại. Người dùng không thể gửi thêm yêu cầu.
* **Giải pháp:** **Đa luồng (Multi-threading)**.
    * **Luồng Printer (Daemon Thread):** Chạy ngầm, liên tục canh chừng hàng đợi để lấy việc làm.
    * **Luồng Main (User Thread):** Giả lập người dùng gửi yêu cầu liên tục mà không bị chặn bởi tốc độ rùa bò của máy in.

### B. Vấn đề giới hạn bộ nhớ (Bounded Buffer)
* **Thực tế:** RAM của máy in có hạn. Nếu hàng đợi nhận vô tận job, hệ thống sẽ tràn bộ nhớ (Crash).
* **Giải pháp:** Sử dụng **Queue có kích thước cố định (`maxsize`)**.
    * **Cơ chế:** Kiểm tra `Full` trước khi thêm.
    * **Hành động:** Nếu đầy -> **DROP (Hủy)** gói tin ngay lập tức để bảo vệ hệ thống.

## 3. Luồng xử lý dữ liệu (Data Flow)

### Bước 1: Khởi tạo Job (Class `PrintJob`)
Đóng gói thông tin vào một Object:
* ID
* Người gửi (Owner)
* Số trang (Pages)

### Bước 2: Logic Người gửi (Producer)
1.  Tạo một Job ngẫu nhiên.
2.  Cố gắng đẩy vào hàng đợi (`queue.put`).
3.  **Xử lý DROP:**
    * Dùng hàm `put_nowait()` hoặc block `try...except Full`.
    * Nếu Queue còn chỗ: Thêm vào -> Báo thành công.
    * Nếu Queue đầy: Báo lỗi "Server Busy" -> Hủy Job.

### Bước 3: Logic Máy in (Consumer)
Chạy vòng lặp vô tận (hoặc check cờ `stop_event`):
1.  **Lấy:** `queue.get()`. Nếu hàng rỗng thì đứng chờ.
2.  **Xử lý:** `time.sleep(random 3-5s)` mô phỏng thời gian in thực tế.
3.  **Báo cáo:** `queue.task_done()` để báo cho hàng đợi biết đã giải phóng 1 slot.

## 4. Tại sao chọn thư viện `queue.Queue`?
Trong Python, tuyệt đối **không dùng `List` (`[]`)** cho bài toán này vì List không an toàn luồng (Not Thread-safe).
* **`queue.Queue`**: Đã tích hợp sẵn cơ chế **Locking (Khóa)**.
* Nó đảm bảo khi Máy in đang lấy Job A ra, thì Người dùng không thể ghi đè lung tung vào vị trí đó. Giúp tránh lỗi Race Condition (Xung đột dữ liệu).

## 5. Bảng vết mô phỏng (Trace Scenario)
Giả sử Queue Max = 3 slot.

| Thời gian | Hành động | Trạng thái Queue | Kết quả |
| :--- | :--- | :--- | :--- |
| **T=0s** | User gửi Job 1 | `[1]` | Job 1 vào chờ. Máy in bắt đầu lấy Job 1 in (cần 5s). |
| **T=1s** | User gửi Job 2 | `[2]` | Job 2 vào chờ (Job 1 đang in bên ngoài). |
| **T=2s** | User gửi Job 3 | `[2, 3]` | Job 3 vào chờ. |
| **T=3s** | User gửi Job 4 | `[2, 3, 4]` | Job 4 vào chờ. **QUEUE FULL (3/3)**. |
| **T=4s** | User gửi Job 5 | `[2, 3, 4]` | **DROP Job 5**. Do hàng đầy, Job 5 bị hủy. |
| **T=5s** | Máy in xong Job 1 | `[3, 4]` | Job 1 xong. Máy in lấy Job 2 ra in. Queue trống 1 slot. |
| **T=6s** | User gửi Job 6 | `[3, 4, 6]` | Job 6 vào thành công (vì vừa có chỗ trống). |

In [1]:
import threading
import time
import random
import queue
from datetime import datetime

# --- 1. ĐỊNH NGHĨA ĐỐI TƯỢNG PRINT JOB ---
class PrintJob:
    def __init__(self, job_id, owner, pages):
        self.job_id = job_id
        self.owner = owner
        self.pages = pages
        self.created_at = datetime.now().strftime("%H:%M:%S")

    def __str__(self):
        return f"[Job #{self.job_id} | {self.owner} | {self.pages} trang | Time: {self.created_at}]"

# --- 2. MÔ PHỎNG MÁY IN (CONSUMER) ---
def printer_daemon(spool_queue, stop_event):
    """
    Hàm này chạy trong một luồng riêng, giả lập máy in đang hoạt động.
    """
    print("\n🖨️  MÁY IN ĐÃ KHỞI ĐỘNG VÀ ĐANG CHỜ LỆNH...\n")
    
    while not stop_event.is_set() or not spool_queue.empty():
        try:
            # Lấy job ra khỏi hàng đợi (chờ tối đa 1s nếu rỗng để check stop_event)
            current_job = spool_queue.get(timeout=1)
            
            # --- BẮT ĐẦU IN ---
            print(f"⚙️  ĐANG IN: {current_job}...")
            
            # Giả lập thời gian in (3-5 giây như đề bài)
            processing_time = random.uniform(3, 5) 
            time.sleep(processing_time)
            
            # --- IN XONG ---
            print(f"✅ HOÀN THÀNH: {current_job} - Mất {processing_time:.2f}s")
            
            # Báo cho hàng đợi biết job này đã xong
            spool_queue.task_done()
            
        except queue.Empty:
            continue

# --- 3. MÔ PHỎNG NGƯỜI DÙNG GỬI YÊU CẦU (PRODUCER) ---
def simulate_lan_traffic(max_queue_size):
    """
    Hàm mô phỏng môi trường LAN với 2 trường hợp bộ nhớ.
    """
    
    # KHI MAX_QUEUE_SIZE = 0 hoặc None -> Hàng đợi vô hạn
    # KHI MAX_QUEUE_SIZE = 10 -> Hàng đợi giới hạn
    if max_queue_size:
        spool_queue = queue.Queue(maxsize=max_queue_size)
        print(f"\n=== BẮT ĐẦU MÔ PHỎNG: BỘ NHỚ GIỚI HẠN ({max_queue_size} SLOT) ===")
    else:
        spool_queue = queue.Queue() # Vô hạn
        print(f"\n=== BẮT ĐẦU MÔ PHỎNG: BỘ NHỚ KHÔNG GIỚI HẠN ===")

    # Tạo sự kiện để dừng máy in khi hết mô phỏng
    stop_event = threading.Event()

    # Khởi động máy in (Chạy song song)
    printer_thread = threading.Thread(target=printer_daemon, args=(spool_queue, stop_event))
    printer_thread.start()

    # Giả lập 20 yêu cầu gửi đến liên tục
    users = ["Admin", "HR_Dept", "Dev_Team", "Manager", "Reception"]
    
    for i in range(1, 21):
        # Tạo ngẫu nhiên số trang từ 1-50
        pages = random.randint(1, 50)
        user = random.choice(users)
        new_job = PrintJob(i, user, pages)

        print(f"📩 Incoming: {new_job.owner} muốn in {pages} trang...")

        try:
            # put_nowait: Thử bỏ vào hàng đợi, nếu đầy thì báo lỗi NGAY LẬP TỨC (không chờ)
            spool_queue.put_nowait(new_job)
            print(f"   -> Đã vào hàng đợi (Queue size: {spool_queue.qsize()})")
        
        except queue.Full:
            # ĐÂY LÀ LOGIC DROP KHI BỘ NHỚ ĐẦY
            print(f"   ❌ TỪ CHỐI: Hàng đợi đã đầy! Job #{i} bị hủy (DROP).")

        # Giả lập các yêu cầu đến rất nhanh (0.5 giây 1 yêu cầu)
        # Nhanh hơn tốc độ in (3-5s) để ép hàng đợi bị đầy
        time.sleep(0.5)

    # Chờ cho đến khi hàng đợi rỗng thì tắt máy in
    spool_queue.join()
    stop_event.set()
    printer_thread.join()
    print("\n=== KẾT THÚC MÔ PHỎNG ===")

# --- 4. CHẠY THỬ NGHIỆM ---

if __name__ == "__main__":
    # --- KỊCH BẢN 1: BỘ NHỚ NHỎ (Sẽ thấy Job bị Drop) ---
    # Ta gửi 20 jobs, nhưng bộ nhớ chỉ chứa 10, in thì chậm -> Chắc chắn sẽ có Drop
    simulate_lan_traffic(max_queue_size=5) 
    # Lưu ý: Tôi để max=5 thay vì 10 để bạn dễ nhìn thấy kết quả Drop nhanh hơn 
    # (vì chờ in 3-5s khá lâu). Bạn có thể sửa thành 10.

    # --- KỊCH BẢN 2: BỘ NHỚ LỚN (Bỏ comment dòng dưới để chạy) ---
    # print("\n\nWait 5s switching scenario...\n")
    # time.sleep(5)
    # simulate_lan_traffic(max_queue_size=0)


=== BẮT ĐẦU MÔ PHỎNG: BỘ NHỚ GIỚI HẠN (5 SLOT) ===

🖨️  MÁY IN ĐÃ KHỞI ĐỘNG VÀ ĐANG CHỜ LỆNH...

📩 Incoming: Admin muốn in 5 trang...
   -> Đã vào hàng đợi (Queue size: 1)
⚙️  ĐANG IN: [Job #1 | Admin | 5 trang | Time: 07:31:02]...
📩 Incoming: HR_Dept muốn in 28 trang...
   -> Đã vào hàng đợi (Queue size: 1)
📩 Incoming: Admin muốn in 10 trang...
   -> Đã vào hàng đợi (Queue size: 2)
📩 Incoming: Dev_Team muốn in 40 trang...
   -> Đã vào hàng đợi (Queue size: 3)
📩 Incoming: Dev_Team muốn in 30 trang...
   -> Đã vào hàng đợi (Queue size: 4)
📩 Incoming: Admin muốn in 46 trang...
   -> Đã vào hàng đợi (Queue size: 5)
📩 Incoming: Reception muốn in 34 trang...
   ❌ TỪ CHỐI: Hàng đợi đã đầy! Job #7 bị hủy (DROP).
📩 Incoming: Manager muốn in 3 trang...
   ❌ TỪ CHỐI: Hàng đợi đã đầy! Job #8 bị hủy (DROP).
✅ HOÀN THÀNH: [Job #1 | Admin | 5 trang | Time: 07:31:02] - Mất 3.91s
⚙️  ĐANG IN: [Job #2 | HR_Dept | 28 trang | Time: 07:31:03]...
📩 Incoming: HR_Dept muốn in 44 trang...
   -> Đã vào hàng đ